<a href="https://colab.research.google.com/github/proywm/PDC-concepts-LiveCoding/blob/main/Live_coding_Data_Locality_ArrayVsLinkedList_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Locality, in the context of computer science, refers to the tendency of programs to access a specific set of data locations within a short period of time. This principle can be broadly categorized into two types: temporal locality and spatial locality. Temporal locality suggests that if a piece of data is accessed once, it is likely to be accessed again soon. Spatial locality implies that if a piece of data is accessed, data nearby in memory is likely to be accessed soon as well. These principles are not limited to computing; they can be observed in real-world scenarios. For example, in a kitchen, frequently used utensils are kept within easy reach, leveraging temporal locality, while ingredients for a recipe are grouped together in a pantry, taking advantage of spatial locality. Similarly, in data centers, computations are often performed close to where the data is stored to reduce access time and improve efficiency, illustrating locality in practice.



**Locality in Computer Architecture**


In computer architecture, locality plays a pivotal role in designing efficient memory hierarchies and caching strategies. Cache memory, positioned between the CPU and main memory, relies heavily on locality principles to speed up data retrieval. Temporal locality helps caches keep recently used data close to the processor, reducing access time for subsequent uses. Spatial locality ensures that blocks of data stored near each other in memory are loaded into the cache together, optimizing the use of cache lines and minimizing cache misses. By leveraging locality, modern processors can significantly reduce the latency associated with memory accesses, thereby enhancing the performance of applications.

**Tutorial Overview: Understanding Locality Through Data Structures**
This tutorial introduces students to the concept of locality and demonstrates how the choice of data structures—arrays versus linked lists—impacts locality and caching behavior, consequently affecting application performance. Students will explore how data is stored and accessed in memory and observe the effects of these accesses on cache performance. Using a live coding environment, the tutorial simulates memory and cache interactions, illustrating the differences in access patterns and locality benefits between arrays and linked lists.

The provided code sets up a memory and cache simulation, initializes data structures, and allows students to visualize how different data structures interact with the cache. By iterating through searches in an array and a linked list, students will see firsthand how spatial and temporal locality influence cache performance. The live demonstration, with step-by-step execution, will help students grasp the importance of locality in computer architecture and its impact on performance optimization.


In [ ]:
!pip install matplotlib


In [1]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import random
from IPython.display import display, clear_output
import ipywidgets as widgets

# Define the memory and cache sizes
MEMORY_SIZE = 32
CACHE_SIZE = 8

# Initialize memory
memory = [random.randint(1, 100) for _ in range(MEMORY_SIZE)]

class Cache:
    def __init__(self, size):
        self.size = size
        self.cache = ["⬜" for _ in range(size)]
        self.cache_miss_count = 0
        self.memory = None

    def initialize_memory(self, memory):
        self.memory = memory

    def load_row_into_cache(self, address):
        row_start = (address // self.size) * self.size
        row_end = row_start + self.size
        self.cache = self.memory[row_start:row_end] + ["⬜"] * (self.size - (row_end - row_start))
        self.cache_miss_count += 1

    def access_data(self, address, data):
        if not self.is_in_cache(data):
            self.load_row_into_cache(address)
        return self.get_cache_index(address)

    def is_in_cache(self, data):
        return data in self.cache

    def get_cache_index(self, address):
        return address % self.size

    def visualize_memory_cache(self, target, memory_index=None, cache_index=None):
        def format_cell(cell, is_target, is_accessed):
            color = "1;31" if is_target else ("1;32" if is_accessed else "0")
            return f"\033[{color}m{cell:02d}\033[0m" if isinstance(cell, int) else cell

        print("Memory:")
        for i in range(0, MEMORY_SIZE, 8):
            print(" ".join(format_cell(cell, cell == target, memory_index == idx) for idx, cell in enumerate(self.memory[i:i+8], i)))

        print("\nCache:")
        print(" ".join(format_cell(cell, cell == target, cache_index == idx) for idx, cell in enumerate(self.cache)))
        print(f"\nCache Miss Count: {self.cache_miss_count}")
        print()

# Function to print linked list
def print_linked_list(head, current_index):
    current = head
    index = 0
    print("Linked List:")
    while current:
        if index == current_index:
            print(f"\033[1;32m{current.data:02d}\033[0m", end=" -> ")
        else:
            print(f"{current.data:02d}", end=" -> ")
        current = current.next
        index += 1
    print("None")

# Define Node class for Linked List
class Node:
    def __init__(self, data, address):
        self.data = data
        self.address = address
        self.next = None

# Create a linked list from memory
def create_linked_list(memory):
    addresses = list(range(MEMORY_SIZE))
    random.shuffle(addresses)
    head = Node(memory[addresses[0]], addresses[0])
    current = head
    for addr in addresses[1:]:
        current.next = Node(memory[addr], addr)
        current = current.next
    return head

class LinkedListSearcher:
    def __init__(self, linked_list_head, cache, target):
        self.linked_list_head = linked_list_head
        self.cache = cache
        self.target = target
        self.current_node = linked_list_head
        self.index = 0

    def iterate_search_linked_list(self, button):
        clear_output(wait=True)

        if self.current_node is None:
            print(f"{self.target} not found in linked list")
            return

        print_linked_list(self.linked_list_head, self.index)

        cache_index = self.cache.access_data(self.current_node.address, self.current_node.data)

        self.cache.visualize_memory_cache(self.target, self.current_node.address, cache_index)

        if self.current_node.data == self.target:
            print(f"Found {self.target} in linked list at index {self.index}")
            return

        self.current_node = self.current_node.next
        self.index += 1
        display(button)

class ArraySearcher:
    def __init__(self, array, cache, target):
        self.array = array
        self.cache = cache
        self.target = target
        self.index = 0

    def iterate_search_array(self, button):
        clear_output(wait=True)

        if self.index >= len(self.array):
            print(f"{self.target} not found in array")
            return

        cache_index = self.cache.access_data(self.index, self.array[self.index])

        self.cache.visualize_memory_cache(self.target, self.index, cache_index)

        if self.array[self.index] == self.target:
            print(f"Found {self.target} in array at index {self.index}")
            return

        self.index += 1
        display(button)

# Initialize cache
cache = Cache(CACHE_SIZE)
cache.initialize_memory(memory)

# Create linked list and initialize searchers
linked_list_head = create_linked_list(memory)
target = memory[10]

linked_list_searcher = LinkedListSearcher(linked_list_head, cache, target)
array_searcher = ArraySearcher(memory, cache, target)

# Create buttons to iterate through search
button_linked_list = widgets.Button(description="Next Step (Linked List)")
button_linked_list.on_click(linked_list_searcher.iterate_search_linked_list)

button_array = widgets.Button(description="Next Step (Array)")
button_array.on_click(array_searcher.iterate_search_array)

# Display initial state and buttons
cache.visualize_memory_cache(None)
print_linked_list(linked_list_head, linked_list_searcher.index)
display(button_linked_list)

print("\nArray Search Initialization:")
cache.visualize_memory_cache(None)
display(button_array)


Memory:
88 37 22 66 06 50 56 05
70 80 17 74 20 22 48 03
27 89 50 14 91 86 45 12
36 09 99 51 73 18 79 64

Cache:
70 80 17 74 20 22 48 03

Cache Miss Count: 2

Found 17 in array at index 10
